In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn import preprocessing
from __future__ import print_function
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC # "Support vector classifier"
from sklearn import mixture
from sklearn.linear_model import LogisticRegression
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import random
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
Bill_Last6Months = pd.read_csv("data/Bill_Last6Months.csv")
Bill_Last6Months_m = pd.read_csv("data/Bill_Last6Months_m.csv")

In [13]:
Bill_Last6Months

,ID,Month,Bill_Amount
0,X000001,7/1/2021,3857.0
1,X000001,8/1/2021,3176.0
2,X000001,9/1/2021,605.0
3,X000001,10/1/2021,0.0
4,X000001,11/1/2021,0.0
...,...,...,...
179995,X030000,8/1/2021,48982.0
179996,X030000,9/1/2021,49860.0
179997,X030000,10/1/2021,36464.0
179998,X030000,11/1/2021,32404.0


In [3]:
Bill_Last6Months_m

,ID,1M_BA,2M_BA,3M_BA,4M_BA,5M_BA,6M_BA,default payment in Jan 2022
0,X000001,3857,3176,605,0,0,0.00,0
1,X000002,2634,1763,2707,3183,3429,3170.00,1
2,X000003,29319,14076,13546,14373,15004,15558.00,0
3,X000004,46925,48279,49337,28236,28977,29641.00,0
4,X000005,8542,5573,35888,20951,19194,19116.00,0
...,...,...,...,...,...,...,...,...
29858,X029996,188893,192825,208314,88020,31150,15968.00,0
29859,X029997,1638,1766,3585,8982,5153,0.00,0
29860,X029998,3598,3307,2755,20862,20497,19321.00,0
29861,X029999,-1561,78437,76238,52802,11937,477.48,0


In [4]:
# Normalizing data
min_max_scaler = preprocessing.MinMaxScaler()
for i in range(1,7): # "1M_BA" to "6M_BA" columns
    Bill_Last6Months_m.iloc[:,i] = pd.DataFrame(min_max_scaler.fit_transform(Bill_Last6Months_m.iloc[:,i].values.reshape(-1, 1)))  

In [5]:
# Diving data to its values and labels
X = Bill_Last6Months_m.iloc[:, 1:10].values
X0 = Bill_Last6Months_m[Bill_Last6Months_m['default payment in Jan 2022'] == 0].iloc[:, 1:6].values
X1 = Bill_Last6Months_m[Bill_Last6Months_m['default payment in Jan 2022'] == 1].iloc[:, 1:6].values

y= Bill_Last6Months_m.iloc[:, -1].values
y0 =  Bill_Last6Months_m[Bill_Last6Months_m['default payment in Jan 2022'] == 0].iloc[:, -1].values
y1 =  Bill_Last6Months_m[Bill_Last6Months_m['default payment in Jan 2022'] == 1].iloc[:, -1].values

In [6]:
#Spliting data to train and test sets
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1)

In [7]:
C0_trs = len(y0_train) # class 0 train size
C0_tes = len(y0_test)  # class 0 test size

C1_trs = len(y1_train) # class 1 train size
C1_tes = len(y1_test)  # class 1 test size

#Shuffle class 0 
train_ri0 = np.random.choice(C0_trs, size=C1_trs, replace=False) #Class 0 Train random_indices
test_ri0 = np.random.choice(C0_tes, size=C1_tes, replace=False) #Class 0 Test random_indices

#Making balanced training set
X_train = np.concatenate((X0_train[train_ri0,:],X1_train), axis=0)
y_train = np.concatenate((y0_train[train_ri0],y1_train), axis=0)

#Making balanced test set
X_test = np.concatenate((X0_test[test_ri0,:],X1_test), axis=0)
y_test = np.concatenate((y0_test[test_ri0],y1_test), axis=0)

In [8]:
#Defining SVM kernel ()
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

model = SVC(kernel = kernels[2])
#Training the model
model.fit(X_train, y_train)

#Testing the model
y_pred = model.predict(X_test)
print('SVM:\n\n',classification_report(y_test,y_pred))

SVM:

               precision    recall  f1-score   support

           0       0.53      0.28      0.37       829
           1       0.51      0.75      0.61       829

    accuracy                           0.52      1658
   macro avg       0.52      0.52      0.49      1658
weighted avg       0.52      0.52      0.49      1658



In [9]:
LR = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr').fit(X_train, y_train)
# solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], default='lbfgs'
y_pred = LR.predict(X_test)

#Reporting the results
print('LOGISTIC Regression:\n\n',classification_report(y_test,y_pred))

LOGISTIC Regression:

               precision    recall  f1-score   support

           0       0.49      0.33      0.40       829
           1       0.50      0.66      0.57       829

    accuracy                           0.49      1658
   macro avg       0.49      0.49      0.48      1658
weighted avg       0.49      0.49      0.48      1658



In [10]:
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
#Reporting the results
print('Random Forest:\n\n',classification_report(y_test,y_pred))

Random Forest:

               precision    recall  f1-score   support

           0       0.52      0.70      0.60       829
           1       0.55      0.36      0.44       829

    accuracy                           0.53      1658
   macro avg       0.54      0.53      0.52      1658
weighted avg       0.54      0.53      0.52      1658



In [11]:
gmm_model = mixture.GaussianMixture(n_components=2, covariance_type='diag') #['spherical', 'diag', 'tied', 'full']
gmm_model.fit(X_train)
y_pred = gmm_model.predict(X_test)

#Reporting the results
Precision = precision_score(y_test, y_pred)
Recall = recall_score(y_test, y_pred)
F1_score = f1_score(y_test, y_pred)
Accuracy = accuracy_score(y_test, y_pred)
print('GMM Results:\n\n',classification_report(y_test,y_pred))

GMM Results:

               precision    recall  f1-score   support

           0       0.50      0.57      0.53       829
           1       0.50      0.43      0.46       829

    accuracy                           0.50      1658
   macro avg       0.50      0.50      0.49      1658
weighted avg       0.50      0.50      0.49      1658



In [12]:
NN = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(5, 2), activation='relu', random_state=1, batch_size= 16, max_iter =10000)
#solver = ['lbfgs', 'sgd', 'adam'], default='adam'
#activation = ['identity', 'logistic', 'tanh', 'relu'], default=’relu’
NN.fit(X_train, y_train)
y_pred = NN.predict(X_test)

#Reporting the results
print('MLP Results:\n\n',classification_report(y_test,y_pred))

MLP Results:

               precision    recall  f1-score   support

           0       0.52      0.22      0.30       829
           1       0.50      0.80      0.62       829

    accuracy                           0.51      1658
   macro avg       0.51      0.51      0.46      1658
weighted avg       0.51      0.51      0.46      1658

